# Big Data Platform
## Assignment 2: MapReduce

**The goal of this assignment is to:**
- Understand and practice the details of MapReduceEngine

**Instructions:**
- Students will form teams of two people each, and submit a single homework for each team.
- The same score for the homework will be given to each member of your team.
- Your solution is in the form of a Jupyter notebook file (with extension ipynb).
- Images/Graphs/Tables should be submitted inside the notebook.
- The notebook should be runnable and properly documented. 
- Please answer all the questions and include all your code.
- You are expected to submit a clear and pythonic code.
- You can change functions signatures/definitions.

**Submission:**
- Submission of the homework will be done via Moodle by uploading a Jupyter notebook.
- The homework needs to be entirely in English.
- The deadline for submission is on Moodle.
- Late submission won't be allowed.
  
  
- In case of identical code submissions - both groups will get a Zero. 
- Some groups might be selected randomly to present their code.

**Requirements:**  
- Python 3.6 should be used.  
- You should implement the algorithms by yourself using only basic Python libraries (such as numpy,pandas,etc.)

<br><br><br><br>

**Grading:**
- Q1 - 5 points - Initial Steps
- Q2 - 50 points - MapReduceEngine
- Q3 - 30 points - Implement the MapReduce Inverted index of the JSON documents
- Q4 - 5 points - Testing Your MapReduce
- Q5 - 10 points - Final Thoughts 

`Total: 100`

**Prerequisites**

In [1]:
# example
!pip install --quiet zipfile36

**Imports**

In [2]:
# general
import os
import time
import random
import warnings
import threading # you can use easier threading packages

# ml
import numpy as np
import scipy as sp
import pandas as pd

# visual
import seaborn as sns
import matplotlib.pyplot as plt

# notebook
from IPython.display import display

**Hide Warnings**

In [3]:
warnings.filterwarnings('ignore')

**Disable Autoscrolling**

In [4]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

**Set Random Seed**

In [5]:
random.seed(123)

<br><br><br><br>
# Question 1
# Initial Steps

Write Python code to create 20 different CSV files in this format:  `myCSV[Number].csv`, where each file contains 10 records. 

The schema is `(‘firstname’,’secondname’,city’)`  

Values should be randomly chosen from the lists: 
- `firstname` : `[John, Dana, Scott, Marc, Steven, Michael, Albert, Johanna]`  
- `city` : `[New York, Haifa, München, London, Palo Alto,  Tel Aviv, Kiel, Hamburg]`  
- `secondname`: any value  

In [6]:
#generates random last names of chosen length
def get_second_name(length):
    import string
    
    upper = string.ascii_uppercase
    lower = string.ascii_lowercase
    first = random.choice(upper)
    partial = ''.join(random.choice(lower) for i in range(length-1))
    
    return first + partial

In [7]:
#function to create the 20 requested csv files
def create_20_files():
    for num in range(20):
        df = pd.DataFrame(index=range(10), columns=range(3))
        df.columns = ['firstname', 'secondname', 'city']
        firstname = ['John', 'Dana', 'Scott', 'Marc', 'Steven', 'Michael', 'Albert', 'Johanna']
        df['firstname'] = np.random.choice(firstname, len(df))
        df['secondname'] = np.random.randint(4, 7, size=len(df))
        df['secondname'] = df['secondname'].apply(lambda x: get_second_name(x))
        city = ['NewYork', 'Haifa', 'Munchen', 'London', 'PaloAlto',  'TelAviv', 'Kiel', 'Hamburg']
        df['city'] = np.random.choice(city, len(df))
        filename = "myCSV" + str(num+1) + ".csv"
        df.to_csv(filename, encoding='utf', index=False)      

In [8]:
#creates the files
create_20_files()

Use python to Create `mapreducetemp` and `mapreducefinal` folders

In [9]:
#function to create folders
def create_folder(pathname):
    if not os.path.exists(pathname):
        os.makedirs(pathname)

In [10]:
#creates folders mapreducetemp and mapreducefinal
create_folder('./mapreducetemp')
create_folder('./mapreducefinal')

<br><br><br>
# Question 2
## MapReduceEngine

Write Python code to create an SQLite database with the following table

`TableName: temp_results`   
`schema: (key:TEXT,value:TEXT)`

In [11]:
import sqlite3

def create_sqlite_db(pathname):
    conn = sqlite3.connect(pathname)
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS temp_results (key TEXT, value TEXT)")

In [12]:
#create sqlite database with table temp_results
create_sqlite_db('./mapreddb')

1. **Create a Python class** `MapReduceEngine` with method `def execute(input_data, map_function, reduce_function)`, such that:
    - `input_data`: is an array of elements
    - `map_function`: is a pointer to the Python function that returns a list where each entry of the form (key,value) 
    - `reduce_function`: is pointer to the Python function that returns a list where each entry of the form (key,value)

<br><br>

**Implement** the following functionality in the `execute(...)` function:

<br>

1. For each key  from the  input_data, start a new Python thread that executes map_function(key) 
<br><br>
2. Each thread will store results of the map_function into mapreducetemp/part-tmp-X.csv where X is a unique number per each thread.
<br><br>
3. Keep the list of all threads and check whether they are completed.
<br><br>
4. Once all threads completed, load content of all CSV files into the temp_results table in SQLite.

    Remark: Easiest way to loop over all CSV files and load them into Pandas first, then load into SQLite  
    `data = pd.read_csv(path to csv)`  
    `data.to_sql(‘temp_results’,sql_conn, if_exists=’append’,index=False)`
<br><br>

5. **Write SQL statement** that generates a sorted list by key of the form `(key, value)` where value is concatenation of ALL values in the value column that match specific key. For example, if table has records
<table>
    <tbody>
            <tr>
                <td style="text-align:center">John</td>
                <td style="text-align:center">myCSV1.csv</td>
            </tr>
            <tr>
                <td style="text-align:center">Dana</td>
                <td style="text-align:center">myCSV5.csv</td>
            </tr>
            <tr>
                <td style="text-align:center">John</td>
                <td style="text-align:center">myCSV7.csv</td>
            </tr>
    </tbody>
</table>

    Then SQL statement will return `(‘John’,’myCSV1.csv, myCSV7.csv’)`  
    Remark: use GROUP_CONCAT and also GROUP BY ORDER BY
<br><br><br>
6. **Start a new thread** for each value from the generated list in the previous step, to execute `reduce_function(key,value)` 
<br>    
7. Each thread will store results of reduce_function into `mapreducefinal/part-X-final.csv` file  
<br>
8. Keep list of all threads and check whether they are completed  
<br>
9. Once all threads completed, print on the screen `MapReduce Completed` otherwise print `MapReduce Failed` 



In [13]:
# implement all of the class here
import threading
import csv

class MapReduceEngine():
    
    
    def store_tmp_results(self, func, data, idx, pathname, params):
            result = func(data, params)
            with open(pathname.format(idx), 'w', encoding='UTF8', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(['key', "value"])
                writer.writerows(result)
                
    def load_csv_to_tempdb(self, pathname):
            conn = sqlite3.connect('./mapreddb')
            data = pd.read_csv('./mapreducetemp/{}'.format(pathname))
            data.to_sql('temp_results', conn, if_exists='append', index=False)
            
    def sorted_list(self):
            query = """
            SELECT key, GROUP_CONCAT(value)
            FROM temp_results
            GROUP BY key
            ORDER BY key
            """
            conn = sqlite3.connect('./mapreddb')
            curs = conn.cursor()
            curs.execute(query)
            return curs.fetchall()
        
    def store_fin_results(self, func, key, val, idx, pathname):
            result = func(key, val)
            with open(pathname.format(idx), 'w', encoding='UTF8', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(['key', 'value'])
                writer.writerows([(result[0], result[1])])
    
    
    def execute(self, input_data, map_function, reduce_function, params):
        
        column_index = params['column'] - 1
        
        m_threads = []
        
        for i in range(len(input_data)): 
            thread = threading.Thread(target=self.store_tmp_results, args=(map_function, 
                                                                      input_data[i], i, 
                                                                      './mapreducetemp/part-tmp-{}.csv',
                                                                      column_index))
            thread.start()
            m_threads.append(thread)
            
        for m_thread in m_threads:
            thread.join()
            
        m_alive = [t for t in m_threads if t.is_alive()]
        
        if len(m_alive) == 0:
            for file in os.listdir("./mapreducetemp"):
                self.load_csv_to_tempdb(file)
        else:
            return ("MapReduce Failed")
        
        s_list = self.sorted_list()
        
        r_threads = []
        
        for i in range(len(s_list)):
            r_thread = threading.Thread(target=self.store_fin_results, args=(reduce_function, 
                                                                           s_list[i][0], 
                                                                           s_list[i][1],
                                                                           i,
                                                                           './mapreducefinal/part-{}-final.csv'))
            
            r_thread.start()
            r_threads.append(r_thread)
    
        for r_thread in r_threads:
            r_thread.join()
        
        r_alive = [rt for rt in r_threads if rt.is_alive()]
        
        if len(r_alive) == 0:
            message = "MapReduce Completed"
        else: 
            message = "MapReduce Failed"
            
        
        return ("MapReduce Completed")
    
    

<br><br><br><br>

# Question 3
## Implement the MapReduce Inverted index of the JSON documents

Implement a function `inverted_map(document_name)` which reads the CSV document from the local disc and return a list that contains entries of the form (key_value, document name).

For example, if myCSV4.csv document has values like:  
`{‘firstname’:’John’,‘secondname’:’Rambo’,‘city’:’Palo Alto’}`

Then `inverted_map(‘myCSV4.csv’)` function will return a list:  
`[(‘firstname_John’,’ myCSV4.csv’),(‘secondname_Rambo’,’ myCSV4.csv’), (‘city_Palo Alto’,’ myCSV4.csv’)]`

In [14]:
def inverted_map(document_name, column_index):
    list_kv = []
    data = pd.read_csv('./{}'.format(document_name))
    for entry in data[data.columns[column_index]].values:
        list_kv.append([entry, document_name])
        
    return list_kv
    

# Write a reduce function `inverted_reduce(value, documents)`, where the field “documents” contains a list of all CSV documents per given value.   
This list might have duplicates.   
Reduce function will return new list without duplicates.

For example,  
calling the function `inverted_reduce(‘firstname_Albert’,’myCSV2.csv, myCSV5.csv,myCSV2.csv’)`   
will return a list `[‘firstname_Albert’,’myCSV2.csv, myCSV5.csv,myCSV2.csv’]`

In [15]:
def inverted_reduce(key, documents):
    docs = documents.split(',')
    no_duplicates = []
    [no_duplicates.append(csv) for csv in docs if csv not in no_duplicates]
    no_dupes = ','.join(no_duplicates)
    
    return [key, no_dupes]

<br><br><br><br>
# Question 4
## Testing Your MapReduce

**Create Python list** `input_data` : `[‘myCSV1.csv’,.. ,‘myCSV20.csv’]`

In [16]:
input_data = ['myCSV{}.csv'.format(i) for i in range(1, 21)]

**Submit MapReduce as follows:**

In [17]:
mapreduce = MapReduceEngine()
status = mapreduce.execute(input_data, inverted_map, inverted_reduce, {"column":1})
print(status)

MapReduce Completed


Make sure that `MapReduce Completed` should be printed and `mapreducefinal` folder should contain the result files.

**Use python to delete all temporary data from mapreducetemp folder and delete SQLite database:**

In [18]:
def delete_tmp_data(path, dbpath):
    for file in os.listdir(path):
        os.remove(path + '/{}'.format(file))
    
    conn = sqlite3.connect(dbpath)
    curs = conn.cursor()
    curs.execute("DROP TABLE temp_results")

In [19]:
delete_tmp_data('./mapreducetemp', './mapreddb')


<br><br><br><br>

# Question 5
# Final Thoughts

The phase where `MapReduceEngine` reads all temporary files generated by maps and sort them to provide each reducer a specific key is called the **shuffle step**.

Please explain **clearly** what would be the main problem of MapReduce when processing Big Data, if there is no shuffle step at all, meaning reducers will directly read responses from the mappers.

Answer:

The shuffle step handles the process of transferring data from mappers to reducers, and makes MapReduce very efficient if implemented correctly.  Map outputs (key, value) pairs, and shuffle ensures that results from mapper that have the same key will be sent to the same reducer, which is necessary for the data to be reduced properly.  Without the shuffle, each reducer would read directly from each mapper, which would contain a mix of different keys. This means that each reducer would have to iterate through the mapper set and would not output a meaningful result.

For example (with shuffle):

(hot, 1)-----mapper 1

(cold, 1)----- mapper 1

(hot, 1) --- mapper 1

(hot, 1)-----mapper 2

(cold, 1) --- mapper 2

(cold, 1)------mapper 2

The shuffle step makes sure that:
(hot,1), (hot,1), (hot,1) are grouped together & given to the same reducer-reducer 1.
(cold,1), (cold,1), (cold,1) are grouped together & given to another reducer-reducer 2.

Then the output from reducer 1 is (hot,3) and reducer 2 is (cold,3) which are easily grouped together as the final output.


Without the shuffle step:
(the results from mapper are the same)

(hot, 1)-----mapper 1

(cold, 1)----- mapper 1

(hot, 1) --- mapper 1

(hot, 1)-----mapper 2

(cold, 1) --- mapper 2

(cold, 1)------mapper 2

The output would directly feed into the reducers without sorting -
(hot,1),(cold, 1), (hot,1) are given to reducer 1 and
(hot,1), (cold,1), (cold, 1) are given to reducer 2

reducer 1 outputs (hot,2), (cold,1) and
reducer 2 outputs (hot, 1), (cold,2)

In this example, the data is only partially reduced.  One would have to initialize an additional sorting step after the reducers to get meaningul results. So in summary, without shuffle/sort, the output from the reducers is either partially reduced or not at all reduced since there is no re-sorting step after reducing.

references: https://datascience.stackexchange.com/questions/11049/why-does-map-reduce-have-a-shuffle-step

<br><br><br><br>
Good Luck :)